In [1]:
import $file.^.SparkInit, SparkInit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/09 16:08:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , SparkInit._

import ss.implicits._

In [3]:
val input_dir = Paths.get(System.getProperty("user.home"), "data", "ot", "extract", "evidence-files")
val df = ss.read.json(input_dir.toString)
df

json at cmd2.sc:2

9 / 9

19/12/09 16:09:17 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


input_dir: java.nio.file.Path = /home/eczech/data/ot/extract/evidence-files
df: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]
res2_2: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]

In [3]:
df.filter($"target.id".contains("uniprot") && $"target.id".contains("-")).select("target.id").show(10, 100)

show at cmd2.sc:1

1 / 1

show at cmd2.sc:1

4 / 4

show at cmd2.sc:1

4 / 4

+---+
| id|
+---+
+---+



In [4]:
df.filter($"target.id".contains("uniprot") && $"target.id".contains("/")).select("target.id").show(10, 100)

show at cmd3.sc:1

1 / 1

show at cmd3.sc:1

4 / 4

+-------------------------------------+
|                                   id|
+-------------------------------------+
|http://identifiers.org/uniprot/P23219|
|http://identifiers.org/uniprot/P35354|
|http://identifiers.org/uniprot/P23219|
|http://identifiers.org/uniprot/P35354|
|http://identifiers.org/uniprot/P10275|
|http://identifiers.org/uniprot/P10275|
|http://identifiers.org/uniprot/P08588|
|http://identifiers.org/uniprot/P08588|
|http://identifiers.org/uniprot/P62942|
|http://identifiers.org/uniprot/P00915|
+-------------------------------------+
only showing top 10 rows



### Gene Data

In [4]:
import java.nio.file.Paths
val path = Paths.get(System.getProperty("user.home"), "data", "ot", "extract", "gene.json")
val df = ss.read.json(path.toString)
df

json at cmd3.sc:3

16 / 16

import java.nio.file.Paths

path: java.nio.file.Path = /home/eczech/data/ot/extract/gene.json
df: org.apache.spark.sql.package.DataFrame = [_private: struct<facets: struct<reactome: struct<pathway_code: array<string>, pathway_type_code: array<string>>>, suggestions: struct<input: array<string>, output: string ... 1 more field>>, alias_name: array<string> ... 60 more fields]
res3_3: org.apache.spark.sql.package.DataFrame = [_private: struct<facets: struct<reactome: struct<pathway_code: array<string>, pathway_type_code: array<string>>>, suggestions: struct<input: array<string>, output: string ... 1 more field>>, alias_name: array<string> ... 60 more fields]

In [5]:
df.withColumn("id_equals", $"id" === $"ensembl_gene_id").groupBy("id_equals").count().show()

show at cmd4.sc:1

16 / 16

show at cmd4.sc:1

1 / 1

+---------+-----+
|id_equals|count|
+---------+-----+
|     true|60564|
+---------+-----+



In [6]:
Seq(
  ("id1", "uid1", Seq("x", "y")),
  ("id2", null, Seq()),
  ("id2", "uid2", Seq())
).toDF("eid", "uid", "uia")
.select($"eid", explode_outer(concat(array($"uid"), $"uia"))).show()

show at cmd5.sc:6

1 / 1

show at cmd5.sc:6

2 / 2

+---+----+
|eid| col|
+---+----+
|id1|uid1|
|id1|   x|
|id1|   y|
|id2|null|
|id2|uid2|
+---+----+



In [7]:
df.filter($"uniprot_id".isNull).count()

count at cmd6.sc:1

16 / 16

count at cmd6.sc:1

1 / 1

res6: Long = 0L

In [8]:
df.withColumn("isBlank", $"uniprot_id" === "").groupBy("isBlank").count().show()

show at cmd7.sc:1

16 / 16

show at cmd7.sc:1

1 / 1

+-------+-----+
|isBlank|count|
+-------+-----+
|   true|40475|
|  false|20089|
+-------+-----+



In [9]:
df.select("uniprot_id", "uniprot_accessions").show(10, 100)

show at cmd8.sc:1

1 / 1

+----------+----------------------------------------+
|uniprot_id|                      uniprot_accessions|
+----------+----------------------------------------+
|          |                                      []|
|    Q8N158|                        [Q8N158, A4D2A7]|
|    Q96RY5|[Q9P2C1, A8MZL1, Q96RY5, B1AJY1, Q8NDN1]|
|    B1AL46|                        [A6NHL0, B1AL46]|
|    P49116|        [B6ZGT8, A8K3H5, P49116, P55092]|
|          |                                      []|
|    O15263|                                [O15263]|
|    Q70SY1|                [Q6P454, Q70SY1, Q6ZMR6]|
|    Q8NGZ5|                [Q5JQT2, Q6IEZ0, Q8NGZ5]|
|    Q8NH54|                [Q8NH54, A6NN77, Q6IFF7]|
+----------+----------------------------------------+
only showing top 10 rows



In [10]:
val dfid = df
    .select($"ensembl_gene_id", explode_outer(
        concat(array($"uniprot_id"), $"uniprot_accessions")
    ).as("uniprot_id"))
    .withColumn("uniprot_id", when(trim($"uniprot_id") =!= "", $"uniprot_id").otherwise(lit(null: String)))
    .dropDuplicates()
dfid.show(10, 100)

show at cmd9.sc:7

16 / 16

show at cmd9.sc:7

1 / 1

+---------------+----------+
|ensembl_gene_id|uniprot_id|
+---------------+----------+
|ENSG00000259490|      null|
|ENSG00000213420|    Q8N158|
|ENSG00000213420|    A4D2A7|
|ENSG00000007545|    Q96RY5|
|ENSG00000007545|    Q9P2C1|
|ENSG00000007545|    A8MZL1|
|ENSG00000007545|    B1AJY1|
|ENSG00000007545|    Q8NDN1|
|ENSG00000228570|    B1AL46|
|ENSG00000228570|    A6NHL0|
+---------------+----------+
only showing top 10 rows



dfid: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ensembl_gene_id: string, uniprot_id: string]

In [11]:
dfid.count()

count at cmd10.sc:1

16 / 16

count at cmd10.sc:1

1 / 1

res10: Long = 129053L

In [20]:
dfid.agg(countDistinct("ensembl_gene_id")).show()

show at cmd19.sc:1

16 / 16

show at cmd19.sc:1

200 / 200

show at cmd19.sc:1

1 / 1

+-------------------------------+
|count(DISTINCT ensembl_gene_id)|
+-------------------------------+
|                          60564|
+-------------------------------+



In [21]:
df.agg(countDistinct("ensembl_gene_id")).show()

show at cmd20.sc:1

16 / 16

show at cmd20.sc:1

200 / 200

show at cmd20.sc:1

1 / 1

+-------------------------------+
|count(DISTINCT ensembl_gene_id)|
+-------------------------------+
|                          60564|
+-------------------------------+



## Resources

In [4]:
val path = Paths.get(System.getProperty("user.home"), "data", "ot", "extract", "resources", "genes_with_non_reference_ensembl_ids.tsv")
//val path = Paths.get(System.getProperty("user.home"), "repos", "ot", "data_pipeline", "mrtarget", "resources", "genes_with_non_reference_ensembl_ids.tsv")
val dfa = ss.read
    .format("csv")
    .option("sep", "\t")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(path.toString)
dfa

load at cmd3.sc:8

1 / 1

load at cmd3.sc:8

2 / 2

path: java.nio.file.Path = /home/eczech/data/ot/extract/resources/genes_with_non_reference_ensembl_ids.tsv
dfa: org.apache.spark.sql.package.DataFrame = [gene_symbol: string, ensembl_gene_id: string ... 3 more fields]
res3_2: org.apache.spark.sql.package.DataFrame = [gene_symbol: string, ensembl_gene_id: string ... 3 more fields]

In [5]:
dfa.show(3)

show at cmd4.sc:1

1 / 1

+-----------+---------------+--------+-------------------+------------+
|gene_symbol|ensembl_gene_id|assembly|         chromosome|is_reference|
+-----------+---------------+--------+-------------------+------------+
|     ABCB11|ENSG00000073734|  GRCh38|                  2|           t|
|     ABCB11|ENSG00000276582|  GRCh38|CHR_HSCHR2_1_CTG7_2|           f|
| AC130343.1|ENSG00000274119|  GRCh38| CHR_HSCHR17_1_CTG2|           f|
+-----------+---------------+--------+-------------------+------------+
only showing top 3 rows



In [6]:
val dfau = dfa.select($"ensembl_gene_id".as("eid"), $"gene_symbol").dropDuplicates()
val dfj = df.select(element_at(split($"target.id", "/"), -1).as("eid")).dropDuplicates()
    .join(dfau, Seq("eid"), "left")
dfj.show(3)

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

200 / 200

show at cmd5.sc:4

9 / 9

show at cmd5.sc:4

1 / 1

+---------------+-----------+
|            eid|gene_symbol|
+---------------+-----------+
|ENSG00000119574|       null|
|ENSG00000162782|       null|
|ENSG00000115705|       null|
+---------------+-----------+
only showing top 3 rows



dfau: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [eid: string, gene_symbol: string]
dfj: org.apache.spark.sql.package.DataFrame = [eid: string, gene_symbol: string]

In [7]:
// Show matches for only unique id matches in both sets
dfj.withColumn("m", $"gene_symbol".isNull).groupBy("m").count().show

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

200 / 200

show at cmd6.sc:1

...

show at cmd6.sc:1

100 / 100

show at cmd6.sc:1

75 / 75

+-----+-----+
|    m|count|
+-----+-----+
| true|36745|
|false|  225|
+-----+-----+



In [8]:
// Show matches for ALL rows of evidence, not just unique ids
df.select(element_at(split($"target.id", "/"), -1).as("eid"))
    .join(dfau, Seq("eid"), "left")
    .withColumn("m", $"gene_symbol".isNull).groupBy("m").count().show

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

200 / 200

show at cmd7.sc:3

...

show at cmd7.sc:3

100 / 100

show at cmd7.sc:3

75 / 75

+-----+-------+
|    m|  count|
+-----+-------+
| true|1701376|
|false|  29192|
+-----+-------+

